In [7]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device.type

'cpu'

In [5]:
# Hyper-parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [8]:
# MNIST dataset (images and labels)
train_dataset = datasets.MNIST(
    root="/home/gopalakrishnanm/Projects/Pytorch_tutorial/data/mnist/",
    train=True,
    download=False,
    transform=ToTensor())
test_dataset = datasets.MNIST(
    root="/home/gopalakrishnanm/Projects/Pytorch_tutorial/data/mnist/",
    train=False,
    download=False,
    transform=ToTensor())

In [9]:
# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [25]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [26]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [30]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Resize and move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 200 == 0:
            print("Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}".format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch: [1/5], Step: [200/600], Loss: 0.1600
Epoch: [1/5], Step: [400/600], Loss: 0.2427
Epoch: [1/5], Step: [600/600], Loss: 0.2374
Epoch: [2/5], Step: [200/600], Loss: 0.2397
Epoch: [2/5], Step: [400/600], Loss: 0.2981
Epoch: [2/5], Step: [600/600], Loss: 0.1479
Epoch: [3/5], Step: [200/600], Loss: 0.1320
Epoch: [3/5], Step: [400/600], Loss: 0.3086
Epoch: [3/5], Step: [600/600], Loss: 0.2911
Epoch: [4/5], Step: [200/600], Loss: 0.2045
Epoch: [4/5], Step: [400/600], Loss: 0.1374
Epoch: [4/5], Step: [600/600], Loss: 0.2512
Epoch: [5/5], Step: [200/600], Loss: 0.1482
Epoch: [5/5], Step: [400/600], Loss: 0.1494
Epoch: [5/5], Step: [600/600], Loss: 0.1943


In [31]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Accuracy of the network on the 10000 test images: 89.04 %
